# OpenAI Function Calling


In the notebook will learn steps on 
- defining a function for getting weather information, 
- setting up a function descriptor, 
- creating a message for a chatbot,
- using the OpenAI API to generate a response.

- OpenAI has fine-tuned that `gpt-3.5-turbo-0613` and `gpt-4-0613` models to:
    - Accept additional arguments through which users can pass in descriptions of functions.

    - If it is relevant, return the name of the function to use, along with a JSON object with the appropriate input parameters.

## Step 1: Import Required Libraries


In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## Step 2: Define a Dummy Weather Function


In [3]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

**Explanation:**

- Import `json`: The json module is imported to handle JSON data formatting.
- Function `get_current_weather`: This function is defined to simulate getting current weather information for a given location.
    - Parameters:
        - `location`: The location for which the weather information is requested.
        - `unit`: The unit of temperature, defaulting to "fahrenheit".
    - Weather Information: A dictionary `weather_info` is created with hardcoded values.
    - Return: The function returns the `weather_info` dictionary as a JSON string using `json.dumps()`.

## Step 3: Define Function Metadata

`functions` parameter allows you to define a set of functions that the model can call during a conversation. This parameter is used to provide the model with additional capabilities, such as calling APIs or executing specific functions that extend the model's functionality beyond text generation.

- here we pass a list of function defination

In [4]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

- **Explanation**: This block defines metadata for the `get_current_weather` function. This metadata helps the OpenAI API understand the function's purpose and its parameters.
    - **Name**: Function name.
    - **Description**: Brief description of what the function does. When passed to LLMs, it is used to determine wheater to use this funcrion or not.
    - **Parameters**: Describes the parameters required by the function, including their *types* and *descriptions*. Here, `location` and `unit` are the parameters defined under properties along with their *type* and *description*. The `location` parameter is mandatory here, therefore its under "required".




## Step 4: Create a User Message

In [5]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

- Explanation: Related to our function, we create a list containing a single user message related to weather.
    - Role: Indicates the message is from the user.
    - Content: The user's query asking about the weather in Boston.

## Step 5: Import and Call the OpenAI API


In [6]:
import openai

In [7]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",  #the model
    messages=messages, #passing our messages defined above
    functions=functions #passing the function we defined
)

- Explanation: This block calls the OpenAI API to get a response based on the provided user message and available functions.
    - Model: Specifies the model to use (e.g., "gpt-3.5-turbo-0613").
    - Messages: Passes the user messages.
    - Functions: Passes the function metadata.

In [8]:
print(response)

{
  "id": "chatcmpl-9YCwc31SepEJQ7N3kvdfrCPqQ3nAo",
  "object": "chat.completion",
  "created": 1717939938,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  },
  "system_fingerprint": null
}


We see, the message we get back contains
- "function_call" with our defined function,
          "name": "get_current_weather" 
- "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
 argument location as key and Boston from our message as value.
          
        

In [9]:
#Extract the message part of the response.
response_message = response["choices"][0]["message"]

In [10]:
response_message

<OpenAIObject at 0x7efb895ba270> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
  }
}

In [11]:
#to access the content of the message 
response_message["content"]

In [12]:
#to extract the function call information from the response.
response_message["function_call"]

<OpenAIObject at 0x7efb895d1130> JSON: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
}

In [13]:
#Parse the arguments of the function call.
json.loads(response_message["function_call"]["arguments"])

{'location': 'Boston, MA'}

## Step 7: Call Dummy Weather Function with Parsed Arguments:

In [14]:
args = json.loads(response_message["function_call"]["arguments"])

In [15]:
# its not strictly in json format, if got any error that might be at model end we can add save guards
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

**Explaination:**
- `args`: Store the parsed function call arguments.
- `get_current_weather(args)`: Call the dummy weather function with the arguments.

## Step 8: Make Another API Call Without Function Call:

In [16]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [17]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

In [18]:
print(response)

{
  "id": "chatcmpl-9YCzC9UHfA3KYU5CrKpOjS4zi3Jd8",
  "object": "chat.completion",
  "created": 1717940098,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  },
  "system_fingerprint": null
}


## Step 9: API Call with Different function_call Options:

In [19]:
#function_call is set to "auto" (letting the model decide)
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

{
  "id": "chatcmpl-9YCzDmhCEViCovyKonzXHDXiIqVSG",
  "object": "chat.completion",
  "created": 1717940099,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  },
  "system_fingerprint": null
}


In [20]:
#function="none" (disabling function call)
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-9YCzLFcJvERs8VPC2u5KxcHqAzayt",
  "object": "chat.completion",
  "created": 1717940107,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 77,
    "completion_tokens": 9,
    "total_tokens": 86
  },
  "system_fingerprint": null
}


In [21]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-9YCzRdJZZ7ybZE7QImiWAEaNQ1eDS",
  "object": "chat.completion",
  "created": 1717940113,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Give me a moment. I will find the current weather in Boston for you."
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 16,
    "total_tokens": 98
  },
  "system_fingerprint": null
}


## Step 10: API Call to Trigger Specific Function:

In [22]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-9YCzSR8moAsAYoO1YTyGsWS1AiMHj",
  "object": "chat.completion",
  "created": 1717940114,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 83,
    "completion_tokens": 12,
    "total_tokens": 95
  },
  "system_fingerprint": null
}


## Step 11: API Call to Get Weather and Use Function:

In [32]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-9YD3MCRXxz8cq6snzexoycz6JihLt",
  "object": "chat.completion",
  "created": 1717940356,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 89,
    "completion_tokens": 11,
    "total_tokens": 100
  },
  "system_fingerprint": null
}


In [37]:
#Append the response message to the conversation history.
messages.append(response["choices"][0]["message"])
messages

[{'role': 'user', 'content': "What's the weather like in Boston!"},
 <OpenAIObject at 0x7efb895e6180> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
   }
 },
 {'role': 'function',
  'name': 'get_current_weather',
  'content': '{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'},
 <OpenAIObject at 0x7efb8901a450> JSON: {
   "role": "assistant",
   "content": "The weather in Boston is currently 72 degrees Fahrenheit. It is sunny and windy."
 }]

In [34]:
#Parse the function call arguments and call the dummy weather function.
args = json.loads(response["choices"][0]["message"]['function_call']['arguments'])
observation = get_current_weather(args)

In [35]:
#Append the function result to the conversation history.
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [36]:
#Make another API call to continue the conversation and print the response.
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response)

{
  "id": "chatcmpl-9YD3Wz0uKOq2Mt1nr0oAyPWtgzCTq",
  "object": "chat.completion",
  "created": 1717940366,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The weather in Boston is currently 72 degrees Fahrenheit. It is sunny and windy."
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 77,
    "completion_tokens": 17,
    "total_tokens": 94
  },
  "system_fingerprint": null
}
